Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Orion"
COLLABORATORS = ""

---

# Intro to Data Science

Based on 

- Dr. VanderPlas' book. Prof at University of Washington and Visting Researcher at Google

> *The Python Data Science Handbook* by Jake VanderPlas (O’Reilly). Copyright 2016 Jake VanderPlas, 978-1-491-91205-8.

Code: <br>
https://jakevdp.github.io/PythonDataScienceHandbook/


## Grades

### Midterm stats
Mean 71.9  <br>
Standard Error 4.7  <br>
Median 82  <br>
Mode 84  <br>

Average of Quiz1, Quiz1b, PA1, PA2a

Will review/update grades during Lab (10/18)


# Fancy Indexing

In the previous sections, we saw how to access and modify portions of arrays using 
- simple indices (e.g., ``arr[0]``), 
- slices (e.g., ``arr[:5]``), and 
- Boolean masks (e.g., ``arr[arr > 0]``).

In this section, we'll look at another style of array indexing, known as **fancy indexing**.

https://i.gifer.com/UTwO.gif

Fancy indexing is like the simple indexing we've already seen, but we pass **arrays of indices in place of single scalars**.

This allows us to very quickly access and modify complicated subsets of an array's values.

## Exploring Fancy Indexing

Fancy indexing is conceptually simple: 

it means passing an **array of indices** to access multiple array elements at once.

For example, consider the following array:

In [ ]:
import numpy as np
rand = np.random.RandomState(42)

x = rand.randint(100, size=10)
print(x)

Suppose we want to **access three different elements**. We could do it like this:

In [ ]:
[x[3], x[7], x[4]]

Alternatively, we can pass a single list or array of indices to obtain the same result:

In [ ]:
ind = [3, 7, 4]
x[ind]

When using fancy indexing, the **shape of the result reflects the shape of the *index arrays*** rather than the shape of the *array being indexed*:

In [ ]:
print(x, ind)
ind = np.array([[3, 7],
                [4, 5]])
x[ind]

Fancy indexing also works in multiple dimensions. 

Consider the following array:

In [ ]:
X = np.arange(12).reshape((3, 4))
X

Like with standard indexing, the first index refers to the row, and the second to the column:

In [ ]:
print(X)
row = np.array([0, 1, 2])
col = np.array([2, 1, 3])
X[row, col]

Notice that the first value in the result is ``X[0, 2]``, the second is ``X[1, 1]``, and the third is ``X[2, 3]``.

Here, each row value is matched with each column vector, exactly as we saw in broadcasting of arithmetic operations.

The pairing of indices in fancy indexing **follows all the broadcasting rules** that were mentioned in DS5: Computation on Arrays: Broadcasting.

- Rule 1: If the two arrays differ in their number of dimensions, the shape of the one with **fewer dimensions is *padded* with ones on its leading (left) side**.
- Rule 2: If the shape of the two arrays does not match in any dimension, the array with shape equal to **1 in that dimension is stretched** to match the other shape.
- Rule 3: If in any dimension the sizes disagree and neither is equal to 1, an **error is raised**.

So, for example, if we combine a column vector and a row vector within the indices, we get a two-dimensional result:

In [ ]:
print(row[:, np.newaxis].shape, col.shape, X)
X[row[:, np.newaxis], col]

```
step 0:
We start with row and column shapes
3,1
3,*
```

```
step 1:
Rule 1: If the two arrays differ in their number of dimensions, the shape of the one with fewer dimensions is *padded* with ones on its leading (left) side.
col: 3,* -> 1,3
```

```
step 2:
Rule 2: If the shape of the two arrays does not match in any dimension, the array with shape equal to 1 in that dimension is stretched to match the other shape.
row: 3,1 -> stretched to 3,3 
col: 1,3 -> stretched to 3,3 

```

In [ ]:
X[row[:, np.newaxis], col]

In [ ]:
#Casting manually
col = np.array([2, 1, 3]) #rule 1: (3,) shaped to 1x3
col_b = np.vstack((col, col, col)) #rule 2: broadcast to 3x3
print(col_b) 
print (col.shape, col_b.shape)

In [ ]:
row = np.array([0, 1, 2])
rx = row[:, np.newaxis] #force shape 3x1
row_b = np.hstack((rx, rx, rx)) #broadcasted in step 2 via rule 2 from (3, 1) to (3, 3)
print (rx.shape, row_b.shape)
# print(row_b) 


In [ ]:
#SSDC
print(X[row[:, np.newaxis], col])
print(X[row_b, col_b])

Another example:

In [ ]:
row[:, np.newaxis] * col

It is always important to remember with fancy indexing that the return value reflects the **broadcasted shape of the indices**, rather than the shape of the array being indexed.

## Combined Indexing

For even more powerful operations, fancy indexing can be combined with the other indexing schemes we've seen:

In [ ]:
print(X)

We can combine fancy and simple indices:

In [ ]:
print(X)
X[2, [2, 0, 1]]

We can also combine fancy indexing with slicing:

In [ ]:
print(X)
X[1:, [2, 0, 1]]

And we can combine fancy indexing with masking:

In [ ]:
mask = np.array([1, 0, 1, 0], dtype=bool)
print(X.shape, row[:, np.newaxis].shape, mask.shape)
#Exercise: Based on the shapes above, what is the expcted shape of X[row[:, np.newaxis], mask]?
X[row[:, np.newaxis], mask]

All of these indexing options combined lead to a very flexible set of operations for accessing and modifying array values.

## Example: Selecting Random Points

One common use of fancy indexing is the selection of subsets of rows from a matrix.

For example, we might have an $N$ by $D$ matrix representing $N$ points in $D$ dimensions, such as the following points drawn from a two-dimensional normal distribution:

In [ ]:
mean = [0, 0] #Dx1
cov = [[1, 2], #DxD
       [2, 5]]
X = rand.multivariate_normal(mean, cov, 100)
X.shape #NxD
#What is D? 

Using the plotting tools from Ch 4: Introduction to Matplotlib, we can visualize these points as a scatter-plot:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()  # for plot styling

plt.scatter(X[:, 0], X[:, 1]);

Let's use fancy indexing to select 20 random points. 

We'll do this by first choosing 20 random indices with no repeats, 

and use these indices to select a portion of the original array:

In [ ]:
print(X.shape)
indices = np.random.choice(X.shape[0], 20, replace=False)
indices

In [ ]:
selection = X[indices]  # fancy indexing here. Same as: X[indices, :]
selection.shape

Now to see which points were selected, let's over-plot large circles at the locations of the selected points:

In [ ]:
plt.scatter(X[:, 0], X[:, 1], alpha=0.3)
plt.scatter(selection[:, 0], selection[:, 1],
            facecolor='b', s=150);

This sort of strategy is often used to quickly partition datasets, as is often needed in **train/test splitting** for validation of statistical models (see Book Ch5.3: Hyperparameters and Model Validation), and in sampling approaches to answering statistical questions.

## Modifying Values with Fancy Indexing

Just as fancy indexing can be used to access parts of an array, it can also be used to **modify** parts of an array.

For example, imagine we have an array of indices and we'd like to set the corresponding items in an array to some value:

In [ ]:
x = np.arange(10)
idx = np.array([2, 1, 8, 4])
x[idx] = 99
print(x)

We can use any assignment-type operator for this. For example:

In [ ]:
x[idx] -= 10
print(x)

Notice, though, that repeated indices with these operations can cause some potentially unexpected results. 

Consider the following:

In [ ]:
x = np.zeros(10)
idx = np.array([0,0])
print(x.shape, idx.shape)
x[idx] = [4, 6]
# x[[0, 0]] = [4, 6]
print(x)

Where did the 4 go? 

The result of this operation is to first assign ``x[0] = 4``, followed by ``x[0] = 6``.

The result, of course, is that ``x[0]`` contains the value 6.

Fair enough, but consider this operation:

In [ ]:
x = np.zeros(10)
i = [2, 3, 3, 4, 4, 4]
x[i] += 1
x

You might expect that ``x[3]`` would contain the value 2, and ``x[4]`` would contain the value 3, as this is how many times each index is repeated. 

Why is this not the case?

Conceptually, this is because ``x[i] += 1`` is meant as a shorthand of **``x[i] = x[i] + 1``**. ``x[i] + 1`` is evaluated, and then the result is assigned to the indices in x.

With this in mind, it is **not the augmentation that happens multiple times, but the assignment**, which leads to the rather nonintuitive results.

So what if you want the other behavior where the operation is repeated? 

For this, you can use the **``at()`` method of ufuncs** (available since NumPy 1.8), and do the following:

In [ ]:
x = np.zeros(10)
np.add.at(x, i, 1)
print(x)
#Note that we don't have to assign result back to x! 

In [ ]:
#Exercise: 
# During my quiz, I forgot the at method. How can I achieve the same result via indexing? 
x = np.zeros(10)
i = [2, 3, 3, 4, 4, 4]
#Goal: x = [ 0.  0.  1.  2.  3.  0.  0.  0.  0.  0.]

print(x)

The ``at()`` method does an in-place application of the given operator at the specified indices (here, ``i``) with the specified value (here, 1).

Another method that is similar in spirit is the ``reduceat()`` method of ufuncs, which you can read about in the NumPy documentation.

https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.ufunc.reduceat.html?highlight=reduceat#numpy.ufunc.reduceat

## Example: Binning Data

You can use these ideas to efficiently bin data to create a histogram by hand.

For example, imagine we have 1,000 values and would like to quickly find where they fall within an array of bins.


We could compute it using ``ufunc.at`` like this:

In [ ]:
np.random.seed(42)
x = np.random.randn(100)

# compute a histogram by hand
#create 20 values linspaced -5 to 5
bins = np.linspace(-5, 5, 20)

print(x.shape, bins.shape, counts.shape)

In [ ]:
# find the appropriate bin for each x
# Shift + Tab: Find indices where elements should be inserted to maintain order.
#Find the indices into a sorted array (bins),
#     if the corresponding elements in (x) were inserted before the indices, the order of (bins) would be preserved.
# Which algo would we use to search a sorted array?
i = np.searchsorted(bins, x) 
print(bins, i)
print(x)

In [ ]:
#Exercise:  
# to add 1 to each of these bins
#use a ufunc to allow repeated addition!

#create a vector full of 0s shaped like bins
counts = np.zeros_like(bins) 

print(counts)

#goal: counts = 
# [ 0.  0.  0.  0.  0.  1.  3.  7.  9. 23. 22. 17. 10.  7.  1.  0.  0.  0.
#   0.  0.]

The counts now reflect the number of points within each bin–in other words, a histogram:

In [ ]:
# plot the results
plt.plot(bins, counts, linestyle='steps');

Of course, it would be silly to have to do this each time you want to plot a histogram.
This is why Matplotlib provides the ``plt.hist()`` routine, which does the same in a single line:

```python
plt.hist(x, bins, histtype='step');
```

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()  # for plot styling

plt.hist(x, bins, histtype='step', color='b')

This function will create a nearly identical plot to the one seen here.
To compute the binning, ``matplotlib`` uses the ``np.histogram`` function, which does a very similar computation to what we did before. Let's compare the two here:

In [ ]:
print("NumPy routine:")
%timeit counts, edges = np.histogram(x, bins)

print("Custom routine:")
%timeit np.add.at(counts, np.searchsorted(bins, x), 1)

Our own one-line algorithm is several times faster than the optimized algorithm in NumPy! How can this be?


If you dig into the ``np.histogram`` source code (you can do this in IPython by typing ``np.histogram??``), you'll see that it's quite a bit more involved than the simple search-and-count that we've done; this is because NumPy's algorithm is more flexible, and particularly is designed for better performance when the number of data points becomes large:

In [ ]:
x = np.random.randn(np.int(1e6))
print("NumPy routine:")
%timeit counts, edges = np.histogram(x, bins)

print("Custom routine:")
%timeit np.add.at(counts, np.searchsorted(bins, x), 1)

What this comparison shows is that algorithmic efficiency is almost never a simple question. 

An algorithm **efficient for large datasets will not always be the best choice for small datasets**, and vice versa (see Ch 2.8 [Big-O Notation]).

But the advantage of coding this algorithm yourself is that with an understanding of these basic methods, you could use these building blocks to extend this to do some very interesting custom behaviors.

The key to efficiently using Python in data-intensive applications is knowing about general convenience routines like ``np.histogram`` and when they're appropriate, but also knowing how to make use of lower-level functionality when you need more pointed behavior.

## Summary 
Fancy indexing 
- passing an array of indices to access multiple array elements at once

- via indices
    - ind = [3, 7, 4]; x[ind]

- via slicing
    - X[1:, [2, 0, 1]]

- via broadcasting
    - X[row[:, np.newaxis], col]

- can also be used to modify our array 
    - x[idx] = [4, 6]

- unexpected behaviour might occur during assignment
    - we can use ufunc `at` [or `reduceat`] for in-place execution
    - np.add.at(x, i, 1)

Good lecture on topic:
http://scipy-lectures.org/intro/numpy/array_object.html
    
Example from the link above:

![numpy_fancy_indexing](numpy_fancy_indexing.png)